In [1]:
# 0. Import modules
from src.instruments.vanilla_option import VanillaOption
from src.market_data.market import Market

from src.models.black_scholes.black_scholes_pricing import BlackScholesEngine
from src.models.pde.pde_pricing import PDEPricingEngine
from src.models.monte_carlo.monte_carlo_pricing import MonteCarloEngine

from src.valuation.valuation_request import ValuationRequest

In [2]:
# 1. Create a Market object
market = Market(
    yield_curve=None,  # a yield curve object or mock
    vol_surface_dict={},
    spot_prices={"STOCK_XYZ": 100.0}
)

In [3]:
# 2. Construct vanilla option Instruments
eu_call_1 = VanillaOption(
    strike=80,
    maturity=1.0,        # 1 year
    option_type="call",
    exercise_style="european"
)

eu_call_2 = VanillaOption(
    strike=100,
    maturity=1.0,        # 1 year
    option_type="call",
    exercise_style="european"
)

eu_call_3 = VanillaOption(
    strike=120,
    maturity=1.0,        # 1 year
    option_type="call",
    exercise_style="european"
)

eu_put_1 = VanillaOption(
    strike=80,
    maturity=1.0,        # 1 year
    option_type="put",
    exercise_style="european"
)

eu_put_2 = VanillaOption(
    strike=100,
    maturity=1.0,        # 1 year
    option_type="put",
    exercise_style="european"
)

eu_put_3 = VanillaOption(
    strike=120,
    maturity=1.0,        # 1 year
    option_type="put",
    exercise_style="european"
)

instruments = [eu_call_1, eu_call_2, eu_call_3, eu_put_1, eu_put_2, eu_put_3]

In [4]:
# 3. Create Pricing Engines

bs_engine = BlackScholesEngine(
    interest_rate=0.02,
    volatility=0.20,
    spot_price=market.get_spot_price("STOCK_XYZ"),
    dividend_yield=0.0
)

mc_engine = MonteCarloEngine(
    interest_rate=0.02,
    volatility=0.20,
    spot_price=market.get_spot_price("STOCK_XYZ"),
    dividend_yield=0.0,
    n_paths=1000000,
    n_steps=252
)

pde_engine = PDEPricingEngine(
    interest_rate=0.02,
    volatility=0.20,
    spot_price=market.get_spot_price("STOCK_XYZ"),
    dividend_yield=0.0,
    nx=300,
    nt=252
)

pricing_engines = [bs_engine, mc_engine, pde_engine]

In [11]:
# Print table header
print(f"{'Option Price':<15}{'Pricing Engine':<20}{'S0':<10}{'Sigma':<10}{'r':<10}{'Strike (K)':<15}{'Maturity':<10}{'Option Type':<15}{'Exercise Style':<15}")
print("=" * 120)

# 4. Create a Valuation Request and run valuations
for instrument in instruments:
    # Get parameters
    strike = instrument.strike
    maturity = instrument.maturity
    option_type = instrument.option_type
    exercise_style = instrument.exercise_style
    
    for pricing_engine in pricing_engines:
        # Get parameters
        pricing_engine_name = pricing_engine.__class__.__name__
        r = pricing_engine.r
        sigma = pricing_engine.sigma
        S0 = pricing_engine.S0
        
        valuation_req = ValuationRequest(instrument=instrument, pricer=pricing_engine, market=market)
        fair_value = valuation_req.run_valuation()
        
        # Print data in tabular format
        print(f"{round(fair_value, 4):<15}{pricing_engine_name:<20}{S0:<10}{sigma:<10}{r:<10}{strike:<15}{maturity:<10}{option_type:<15}{exercise_style:<15}")


Option Price   Pricing Engine      S0        Sigma     r         Strike (K)     Maturity  Option Type    Exercise Style 
22.5429        BlackScholesEngine  100.0     0.2       0.02      80             1.0       call           european       
22.5656        MonteCarloEngine    100.0     0.2       0.02      80             1.0       call           european       
22.5428        PDEPricingEngine    100.0     0.2       0.02      80             1.0       call           european       
8.916          BlackScholesEngine  100.0     0.2       0.02      100            1.0       call           european       
8.915          MonteCarloEngine    100.0     0.2       0.02      100            1.0       call           european       
8.9119         PDEPricingEngine    100.0     0.2       0.02      100            1.0       call           european       
2.5469         BlackScholesEngine  100.0     0.2       0.02      120            1.0       call           european       
2.5371         MonteCarloEngine 